All quotes below are excerpted from https://toadstyle.org/cryptopals/60.txt

```
60. Single-Coordinate Ladders and Insecure Twists

All our hard work is about to pay some dividends. Here's a list of
cool-kids jargon you'll be able to deploy after completing this
challenge:

* Montgomery curve
* single-coordinate ladder
* isomorphism
* birational equivalence
* quadratic twist
* trace of Frobenius

Not that you'll understand it all; you won't. But you'll at least be
able to silence crypto-dilettantes on Twitter.

Now, to the task at hand. In the last problem, we implemented ECDH
using a short Weierstrass curve form, like this:

    y^2 = x^3 + a*x + b

For a long time, this has been the most popular curve form. The NIST
P-curves standardized in the 90s look like this. It's what you'll see
first in most elliptic curve tutorials (including this one).

We can do a lot better. Meet the Montgomery curve:

    B*v^2 = u^3 + A*u^2 + u

Although it's almost as old as the Weierstrass form, it's been buried
in the literature until somewhat recently. The Montgomery curve has a
killer feature in the form of a simple and efficient algorithm to
compute scalar multiplication: the Montgomery ladder.

Here's the ladder:

    function ladder(u, k):
        u2, w2 := (1, 0)
        u3, w3 := (u, 1)
        for i in reverse(range(bitlen(p))):
            b := 1 & (k >> i)
            u2, u3 := cswap(u2, u3, b)
            w2, w3 := cswap(w2, w3, b)
            u3, w3 := ((u2*u3 - w2*w3)^2,
                       u * (u2*w3 - w2*u3)^2)
            u2, w2 := ((u2^2 - w2^2)^2,
                       4*u2*w2 * (u2^2 + A*u2*w2 + w2^2))
            u2, u3 := cswap(u2, u3, b)
            w2, w3 := cswap(w2, w3, b)
        return u2 * w2^(p-2)

You are not expected to understand this.
```

In [1]:
from dataclasses import dataclass

In [2]:
from math import log, ceil


def cswap(a, b, i):
    # b,a if i else a,b
    return (b*i + a*(1-i), a*i + b*(1-i))


@dataclass
class MontyCurve:
    a: int
    b: int
    p: int
    
    def mul(self, u, k):
        a, p = self.a, self.p
        blp = int(ceil(log(p, 2)))  # bitlength of p
        u2, w2 = (1, 0)
        u3, w3 = (u, 1)
        for i in (range(blp)[::-1]):
            b = 1 & (k >> i)
            u2, u3 = cswap(u2, u3, b)
            w2, w3 = cswap(w2, w3, b)
            u3, w3 = ((u2*u3 - w2*w3)**2 % p,
                       u * (u2*w3 - w2*u3)**2 % p)
            u2, w2 = ((u2**2 - w2**2)**2 % p,
                       4*u2*w2 * (u2**2 + a*u2*w2 + w2**2) % p)
            u2, u3 = cswap(u2, u3, b)
            w2, w3 = cswap(w2, w3, b)
        return u2 * w2 ** (p-2)

```
Go ahead and implement the ladder. Remember that all computations are
in GF(233970423115425145524320034830162017933).

Oh yeah, the curve parameters. You might be thinking that since we're
switching to a new curve format, we also need to pick out a whole new
curve. But you'd be totally wrong! It turns out that some short
Weierstrass curves can be converted into Montgomery curves.

You can perform this conversion algebraically. But it's kind of a
pain, so here you go:

    v^2 = u^3 + 534*u^2 + u

Through cunning and foresight, I have chosen this curve specifically
to have a really simple map between Weierstrass and Montgomery
forms. Here it is:

    u = x - 178
    v = y

Which makes our base point:

    (4, 85518893674295321206118380980485522083)

Or, you know. Just 4.

Anyway, implement the ladder. Verify ladder(4, n) = 0. Map some points
back and forth between your Weierstrass and Montgomery representations
and verify them.
```

In [ ]:
curve = MontyCurve(a=534, b=1, p=233970423115425145524320034830162017933)
order = 29246302889428143187362802287225875743

print(curve.mul(4, order+1))

In [ ]:
# copied over (lightly edited) from challenge_59.ipynb


class WeierZero:
    pass


class WeierCurve:
    def __init__(self, a, b, p):
        self.zero = WeierZero()
        self.a = a
        self.b = b
        self.p = p
    
    def inv(self, pt):
        x, y = pt
        p = self.p
        return (x, p-y)  # = (x, -y) in GF(p)

    def add(self, p1, p2):  # don't worry about how this works. it's ~magic~
        zero = self.zero
        if p1 is zero: return p2
        if p2 is zero: return p1
        if p1 == self.inv(p2): return zero
        
        a, p = self.a, self.p
        x1, y1 = p1
        x2, y2 = p2
        
        if p1 == p2:
            top = (3 * x1**2 + a) % p
            btm = (2 * y1) % p
        else:
            top = (y2 - y1) % p
            btm = (x2 - x1) % p
        m = (top * invmod(btm, p)) % p
        
        x3 = (m**2 - x1 - x2) % p
        y3 = (m*(x1 - x3) - y1) % p
        return x3, y3
    
    def mul(self, pt, k):
        result = self.zero
        add = self.add
        while k:
            if k & 1:
                result = add(result, pt)
            pt = add(pt, pt)
            k >>= 1
        return result

```
One nice thing about the Montgomery ladder is its lack of special
cases. Specifically, no special handling of: P1 = O; P2 = O; P1 = P2;
or P1 = -P2. Contrast that with our Weierstrass addition function and
its battalion of ifs.

And there's a security benefit, too: by ignoring the v coordinate, we
take away a lot of leeway from the attacker. Recall that the ability
to choose arbitrary (x, y) pairs let them cherry-pick points from any
curve they can think of. The single-coordinate ladder robs the
attacker of that freedom.

But hang on a tick! Give this a whirl:

    ladder(76600469441198017145391791613091732004, 11)
```

In [ ]:
u = 76600469441198017145391791613091732004
curve.mul(u, 11)

```
What the heck? What's going on here?

Let's do a quick sanity check. Here's the curve equation again:

    v^2 = u^3 + 534*u^2 + u

Plug in u and take the square root to recover v.
```

In [ ]:
# helper functions for modular sqrt - copied over from challenge_59.ipynb

class NoQuadraticResidueError(Exception):  pass


def eulers_criterion(n, p):
    # tests whether n is a quadratic residue mod p
    # (i.e. whether there exists x such that pow(x, 2, p) == n)
    return pow(n, (p-1)//2, p) == 1


def tonelli_shanks(n, p):
    if not eulers_criterion(n, p):
        raise NoQuadraticResidueError

    # ref: https://en.wikipedia.org/wiki/Tonelli%E2%80%93Shanks_algorithm

    # 1. find Q, S such that Q is odd and Q * 2**S = p-1
    Q, S = p-1, 0
    while Q & 1 == 0:  # faster than Q % 2 == 0
        Q >>= 1  # faster than Q //= 2
        S += 1

    # 2. find some int z such that z is not a quadratic residue mod p
    z = 2
    while eulers_criterion(z, p):
        z += 1
        assert z < p

    # 3. initialize main loop's state variables
    M = S
    c = pow(z, Q, p)
    t = pow(n, Q, p)
    R = pow(n, (Q+1) // 2, p)

    # 4. main loop
    while t > 1:
        # find i's value using repeated squaring
        t_sq = t
        for i in count(1):
            t_sq = pow(t_sq, 2, p)
            if t_sq == 1:
                break
            assert i < M  # sanity-check: if i >= M the residue doesn't exist
                          # (shouldn't ever happen - we made sure the residue
                          # exists back at the top of this function)

        # update state variables and loop
        exponent = M - i - 1
        if exponent < 0:
            b = pow(c, 2**(-exponent), p)
            b = (b * invmod(c, p)) % p
        else:
            b = pow(c, 2**exponent, p)

        M = i
        c = pow(b, 2, p)
        t = (t * c) % p
        R = (R * b) % p

    if t == 0:
        return 0
    
    res1 = R
    res2 = (-R) % p
    return res1, res2

In [ ]:
def u_to_v(u, curve=curve):
    p = curve.p
    v_sq = (u**3 + curve.a * u**2 + u) % p
    return tonelli_shanks(v_sq, p)

try:
    u_to_v(u)
except NoQuadraticResidueError:
    print("ERROR: Square root of", u, "does not exist!")

```
You should detect that something is quite wrong. This u does not
represent a point on our curve! Not every u does.

This means that even though we can only submit one coordinate, we
still have a little bit of leeway to find invalid
points. Specifically, an input u such that u^3 + 534*u^2 + u is not a
quadratic residue can never represent a point on our curve. So where
the heck are we?

The other curve we're on is a sister curve called a "quadratic twist",
or simply "the twist". There is actually a whole family of quadratic
twists to our curve, but they're all isomorphic to each
other. Remember that that means they have the same number of points,
the same subgroups, etc. So it doesn't really matter which particular
twist we use; in fact, we don't even need to pick one.

...

If Alice chose a curve with an insecure twist, i.e. one with a
partially smooth order, then some doors open back up for Eve. She can
choose low-order points on the twisted curve, send them to Alice, and
perform the invalid-curve attack as before.

The only caveat is that she won't be able to recover the full secret
using off-curve points, only a fraction of it. But we know how to
handle that.

So:

1. Calculate the order of the twist and find its small factors. This
   one should have a bunch under 2^24.
```

In [6]:
# helper function copied from challenge_57.ipynb
# generates primes using a modified, unbounded version of the Sieve of Eratosthenes
def primegen(up_to=None):
    yield 2
    d = {}
    counter = count(3, 2) if up_to is None else range(3, up_to, 2)
    for i in counter:
        l = d.pop(i, None)
        if l:
            for n in l:
                d.setdefault(i+(2*n), []).append(n)
            continue
        yield i
        d.setdefault(3*i, []).append(i)

In [14]:
# The ordinary curve and its twist have 2*p + 2 points between them.
# The ordinary curve's order is known, so we can derive the twist's order easily enough:

twist_order = 2*curve.p + 2 - order
print("Twist's order:", twist_order)
print("Factoring...")
factors = [p for p in primegen(up_to=2**25) if twist_order % p == 0]
print("Small factors:", factors)

Twist's order: 438694543341422147861277267373098160125
Factoring...
Small, nonrepeated factors: [3, 5, 463, 619189]


In [15]:
t = curve.p + 1 - order
print(curve.p + 1 + t)
print(log(twist_order, 2))

438694543341422147861277267373098160125
128.3664843673693


In [16]:
# four factors? that sure doesn't seem like "a bunch"... can that be right?
# TODO WIP